In [27]:
import json
import os
import sys

import pandas as pd
import xgboost as xgb

from classes import Model, Node, Tree, SplitType, ParamT

import time

import json
import os
import pickle

import pandas as pd

import treelite
from treelite import Model, ModelBuilder
import treelite.sklearn

## XGBoost

In [28]:
xgboost_100_path_json = "/home/nano/projects/tree-ensemble-model-management/models/xgboost_100_trees_daily_2016-01-01.json"
xgboost_1000_path_json = "models/xgboost/daily/xgboost_1000_trees_daily_2016-01-01.json"

In [46]:
for file_name in [
    xgboost_100_path_json,
    xgboost_1000_path_json,
]:
    print("JSON File_size: ", os.path.getsize(file_name))

    # Load XGBoost Model
    t0 = time.time()
    with open(file_name, "r") as f:
        model_json = f.read()
    booster = xgb.Booster()
    booster.load_model(file_name)
    print("JSON dt_recover: ", time.time() - t0)

    t0 = time.time()
    booster.save_model("model.json")
    print("JSON dt_save: ", time.time() - t0)

    # Treelite
    t0 = time.time()
    with open(file_name, "r") as f:
        model_json = f.read()
    model = Model.from_xgboost_json(model_json)
    model.serialize("treelite.bin")
    print("Treelite dt_save: ", time.time() - t0)

    t0 = time.time()
    model = Model()
    model.deserialize("treelite.bin")
    print("Treelite dt_recover: ", time.time() - t0)
    print("Treelite File_size: ", os.path.getsize("treelite.bin"))

JSON File_size:  108221
JSON dt_recover:  0.012692928314208984
JSON dt_save:  0.011789083480834961
Treelite dt_save:  0.0049610137939453125
Treelite dt_recover:  0.00041556358337402344
Treelite File_size:  107436
JSON File_size:  1064294
JSON dt_recover:  0.0659949779510498
JSON dt_save:  0.03760528564453125
Treelite dt_save:  0.0257265567779541
Treelite dt_recover:  0.0018093585968017578
Treelite File_size:  1061328


## SKLearn

In [47]:
sklearn_100_path_pkl = "/home/nano/projects/tree-ensemble-model-management/models/sklearn/daily/100_trees/sklearn_100_trees_daily_2016-01-02.pkl"
sklearn_1000_path_pkl = "/home/nano/projects/tree-ensemble-model-management/models/sklearn/daily/sklearn_1000_trees_daily_2016-01-01.pkl"

In [52]:
for file_name in [
    sklearn_100_path_pkl,
    sklearn_1000_path_pkl,
]:
    # Load Pickle Model
    t0 = time.time()
    with open(file_name, "rb") as f:
        model = pickle.load(f)
    print("Pickle dt_recover: ", time.time() - t0)

    # Save Pickle Model
    t0 = time.time()
    with open(file_name + ".pkl", "wb") as f:
        pickle.dump(booster, f)
    print("Pickle dt_save: ", time.time() - t0)

    print("Pickle File_size: ", os.path.getsize(file_name))

    # Treelite Save
    t0 = time.time()
    with open(file_name, "rb") as f:
        model = pickle.load(f)
    model = treelite.sklearn.import_model(model)
    model.serialize("treelite.bin")
    print("Treelite dt_save: ", time.time() - t0)

    # Treelite Deserialize
    t0 = time.time()
    model = Model()
    model.deserialize("treelite.bin")
    print("Treelite dt_recover: ", time.time() - t0)

    print("Treelite File_size: ", os.path.getsize("treelite.bin"))
    
    print()
    print()

Pickle dt_recover:  0.005334138870239258
Pickle dt_save:  0.03096604347229004
Pickle File_size:  133272
Treelite dt_save:  0.010691642761230469
Treelite dt_recover:  0.00045180320739746094
Treelite File_size:  127212


Pickle dt_recover:  0.0289609432220459
Pickle dt_save:  0.022776126861572266
Pickle File_size:  1318739
Treelite dt_save:  0.26085782051086426
Treelite dt_recover:  0.0026466846466064453
Treelite File_size:  1269312


